In [1]:
import re
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold
from transformers import BertForSequenceClassification
from torch.optim import AdamW
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)

## **1. Setting Up the Tokenizer**
Before processing the text data, we modify the DeBERTa tokenizer to recognize entity markers.

## **2. Parsing the Data**
The parse_data function reads a structured text file where:
Since each example consists of 4 lines, we process 4 lines at a time.
- We preprocess the sentences, record the entity tag positions, and remove the tags.
- Then we extract the relationship information and add the data to the list.

For example, the train file.txt looks like below

_10	"The solute was placed inside a beaker and 5 mL of the <e1>solvent</e1> was pipetted into a 25 mL glass <e2>flask</e2> for each trial."_

_Entity-Destination(e1,e2)_

_Comment:_

_(empty)_

## **3. Preprocessing Data for Model Input**

The preprocess_data function:

- Inserts the special entity markers (@E1@ ... @/E1@, @E2@ ... @/E2@) into the sentence.
- Tokenizes the modified sentence using DeBERTa's tokenizer.
- Identifies entity positions in the tokenized sequence.
- Converts relation labels into numerical IDs

## **4. Preparing the Dataset**

- Load the training and validation data from TXT files.
- Extract unique relation labels and create a label map.
- Preprocess both training and validation data to obtain tokenized inputs.

In [3]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base')
special_tokens = ["@E1@", "@/E1@", "@E2@", "@/E2@"]
tokenizer.add_tokens(special_tokens)
def parse_data(file_path):
    '''
    Take the TXT file as input and extract the sentences, relations and entities
    and return it in form of a dictionary
    '''
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    for i in range(0, len(lines), 4):  # Each example spans 4 lines
        # Extract sentence
        sentence = lines[i].split('\t')[1].strip()
        sentence = sentence.replace('< e1 >', '<e1>')
        sentence = sentence.replace('< e2 >', '<e2>')
        sentence = sentence.replace('< /e1 >', '</e1>')
        sentence = sentence.replace('< /e2 >', '</e2>')

        sentence = re.sub(' {2,}', ' ', sentence)
        # Extract entity markers
        e1_start = sentence.find('<e1>')
        e1_end = sentence.find('</e1>')
        e2_start = sentence.find('<e2>')
        e2_end = sentence.find('</e2>')

        # Remove entity markers from the sentence
        sentence = sentence.replace('<e1>', '').replace('</e1>', '').replace('<e2>', '').replace('</e2>', '')
        # if re.match(r"[\*\"\n\\…\+\-\/\=\(\)‘•€\[\]\|♫:;—”“~`#]", sentence):
        #   print(sentence)
        # Extract relation label
        relation = lines[i + 1].strip()

        # Append to data
        data.append({
            'sentence': sentence,
            'e1_start': e1_start,
            'e1_end': e1_end - 4,  # Adjust for removed markers
            'e2_start': e2_start - 4 if e2_start > e1_end else e2_start - 8,
            'e2_end': e2_end - 4 if e2_end > e1_end else e2_end - 8,
            'relation': relation
        })

    return data
def preprocess_data(data, tokenizer, label_map):
    '''
    Takes the data dictionary, tokenizer and label_map and returns the tokenized inputs, attention_mask, position of entity 1,
    position of entiy 2 and the labels
    '''
    processed_data = []

    for item in data:
        sentence = item['sentence']
        e1_start, e1_end = item['e1_start'], item['e1_end']
        e2_start, e2_end = item['e2_start'], item['e2_end']

        # Insert entity markers (using unique symbols for better token recognition)
        sentence = (
            sentence[:e1_start] + ' @E1@ ' + sentence[e1_start:e1_end] + ' @/E1@ ' +
            sentence[e1_end:e2_start] + ' @E2@ ' + sentence[e2_start:e2_end] + ' @/E2@ ' +
            sentence[e2_end:]
        )

        # Tokenize the sentence
        inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

        # Find entity positions in the tokenized sequence
        input_ids = inputs['input_ids'].squeeze(0).tolist()
        entity_pos_1 = next((i for i, id in enumerate(input_ids) if id == tokenizer.convert_tokens_to_ids("@E1@")), -1)
        entity_pos_2 = next((i for i, id in enumerate(input_ids) if id == tokenizer.convert_tokens_to_ids("@E2@")), -1)

        if entity_pos_1 == -1 or entity_pos_2 == -1:
            print(f"Warning: Entity positions not found for sentence: {sentence}")

        # Convert label to ID
        label = label_map[item['relation']]

        processed_data.append({
            'input_ids': torch.tensor(input_ids),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'entity_pos_1': torch.tensor(entity_pos_1),
            'entity_pos_2': torch.tensor(entity_pos_2),
            'label': label
        })

    return processed_data
train_data = parse_data('TRAIN_FILE.TXT')
valid_data = parse_data('TEST_FILE_FULL.TXT')
all_labels = set([item['relation'] for item in train_data] + [item['relation'] for item in valid_data])
label_map = {label: idx for idx, label in enumerate(sorted(all_labels))}
valid_processed = preprocess_data(valid_data, tokenizer, label_map)

In [4]:
def extract_entity_representations(sequence_output, entity_positions, max_len):
    """
    Extracts entity representations by averaging over all tokens in the entity span.

    Args:
        sequence_output: Tensor of shape (batch_size, seq_length, hidden_size)
        entity_positions: Tensor of shape (batch_size, max_entity_len)
        max_len: Maximum number of tokens to consider per entity

    Returns:
        Tensor of shape (batch_size, hidden_size)
    """
    batch_size, _, hidden_size = sequence_output.shape
    entity_embs = torch.zeros(batch_size, hidden_size, device=sequence_output.device)

    for i in range(batch_size):
        positions = entity_positions[i]  # Extract tensor of indices
        if positions.numel() > 0:  # Check if it's non-empty
            positions = positions.tolist()  # Convert tensor to list
            entity_embs[i] = sequence_output[i, positions, :].mean(dim=0)  # Compute mean

    return entity_embs


In [5]:
class FCLayer(nn.Module):
  '''
    Fully Connected Layer with optional dropout, layer normalization, and activation.

    This class implements a standard feedforward neural network layer with:
    - Dropout for regularization.
    - Layer normalization for stable training.
    - An optional activation function (default: Tanh).

    Args:
        input_dim (int): Size of the input feature vector.
        output_dim (int): Size of the output feature vector.
        dropout_rate (float, optional): Dropout probability (default: 0.1).
        use_activation (bool, optional): Whether to apply the activation function (default: True).
        use_norm (bool, optional): Whether to apply layer normalization (default: True).

    Forward Pass:
        x (Tensor): Input tensor of shape `(batch_size, input_dim)`.

    Returns:
        Tensor: Transformed output of shape `(batch_size, output_dim)`.
  '''
  def __init__(self, input_dim, output_dim, dropout_rate=0.1, use_activation=True, use_norm=True):
    super(FCLayer, self).__init__()
    self.use_activation = use_activation
    self.Dropout = nn.Dropout(dropout_rate)
    self.linear = nn.Linear(input_dim, output_dim)
    self.layer_norm = nn.LayerNorm(output_dim)
    self.activation = nn.Tanh()
    self.use_norm = use_norm
  def forward(self, x):
    x = self.Dropout(x)
    if self.use_norm:
      x = self.layer_norm(x)
    if self.use_activation:
      x = self.activation(x)
    return self.linear(x)

class DeBertaEntityBasedClassifier(nn.Module):
  '''
    DeBERTa-based classifier for relation extraction using entity-aware representations.

    This model fine-tunes DeBERTa to classify relationships between two entities in a sentence.
    It uses:
    - A DeBERTa backbone to extract contextualized token embeddings.
    - A fully connected (FC) layer for CLS token processing.
    - FC layers for entity-specific feature extraction.
    - A final FC layer for classification.

    Args:
        model_name (str): Name of the pre-trained DeBERTa model from Hugging Face.
        num_labels (int): Number of relation classes.
        dropout_rate (float, optional): Dropout probability for regularization (default: 0.1).

    Forward Pass:
        input_ids (Tensor): Tokenized input of shape `(batch_size, sequence_length)`.
        attention_mask (Tensor): Attention mask of shape `(batch_size, sequence_length)`.
        entity_pos_1 (Tensor): Positions of the first entity in each sentence `(batch_size,)`.
        entity_pos_2 (Tensor): Positions of the second entity in each sentence `(batch_size,)`.
        labels (Tensor, optional): True labels for computing loss `(batch_size,)`.

    Returns:
        dict:
            - 'logits' (Tensor): Classification logits `(batch_size, num_labels)`.
            - 'loss' (Tensor, optional): Cross-entropy loss if labels are provided.
  '''
  def __init__(self, model_name, num_labels, dropout_rate=0.1):
    super(DeBertaEntityBasedClassifier, self).__init__()
    config = AutoConfig.from_pretrained(model_name)
    self.backbone = AutoModel.from_pretrained(model_name, config=config)

    hidden_size = config.hidden_size
    self.num_labels = num_labels

    self.cls_fc = FCLayer(hidden_size, hidden_size, dropout_rate)
    self.entity_fc = FCLayer(hidden_size, hidden_size, dropout_rate)
    self.final_fc = FCLayer(hidden_size * 3, num_labels, dropout_rate, use_activation=False, use_norm=False)

  def forward(self, input_ids, attention_mask, entity_pos_1, entity_pos_2, labels=None):
    outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
    sequence_output = outputs.last_hidden_state
    #print(f"Sequence output shape: {sequence_output.shape}")
    cls_output = sequence_output[:, 0, :]
    cls_fc = self.cls_fc(cls_output)
    #print(f"CLS output shape: {cls_output.shape}")
    #print(f"CLS FC output shape: {cls_fc.shape}")
    # entity_1_emb = sequence_output[torch.arange(sequence_output.shape[0]), entity_pos_1, :]
    # entity_2_emb = sequence_output[torch.arange(sequence_output.shape[0]), entity_pos_2, :]  # (batch_size, hidden_size)
    entity_1_emb = extract_entity_representations(sequence_output, entity_pos_1, max_len=input_ids.shape[1])
    entity_2_emb = extract_entity_representations(sequence_output, entity_pos_2, max_len=input_ids.shape[1])
    #print(f"Entity 1 embedding shape: {entity_1_emb.shape}")
    #print(f"Entity 2 embedding shape: {entity_2_emb.shape}")
    entity_1_emb = self.entity_fc(entity_1_emb)
    entity_2_emb = self.entity_fc(entity_2_emb)
    #print(f"Entity 1 FC output shape: {entity_1_emb.shape}")
    #print(f"Entity 2 FC output shape: {entity_2_emb.shape}")

    combined = torch.cat([cls_output, entity_1_emb, entity_2_emb], dim=-1)
    #print(f"Combined output shape: {combined.shape}")
    logits = self.final_fc(combined)
    loss = None
    if labels != None:
      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits, labels)
    return {'logits': logits, 'loss': loss}

## Validation Accuracy
- Load the validation data from the file **TEST_FILE_FULL.TXT**
- Process the sentences
- Load the saved model parameters from **best_model_rdeberta-2.pth**
- Calculate accuracy score and F1 score without the class other

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = 'microsoft/deberta-base'
model = DeBertaEntityBasedClassifier(model_name=model_name, num_labels=len(label_map)).to(device)
model.backbone.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load("best_model_rdeberta-2.pth", weights_only=True, map_location=device))

# Process Test Data
test_data = parse_data('TEST_FILE_FULL.TXT')
test_processed = preprocess_data(test_data, tokenizer, label_map)
input_ids = torch.stack([item['input_ids'] for item in test_processed])
attention_masks = torch.stack([item['attention_mask'] for item in test_processed])
labels_test = torch.tensor([item['label'] for item in test_processed])
entity_1_positions = torch.stack([item['entity_pos_1'] for item in test_processed])
entity_2_positions = torch.stack([item['entity_pos_2'] for item in test_processed])

test_dataset = TensorDataset(input_ids, attention_masks, labels_test, entity_1_positions, entity_2_positions)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # Don't shuffle test data

model.eval() # Set the model to evaluation mode to stop dropout



predictions, true_labels = [], []
with torch.no_grad():  # Disable gradient computation
  for batch in test_loader:
      input_ids, attention_mask, labels, entity_1_positions, entity_2_positions = batch
      input_ids = input_ids.to(device)
      attention_mask = attention_mask.to(device)
      labels = labels.to(device)
      entity_1_positions = entity_1_positions.to(device)
      entity_2_positions = entity_2_positions.to(device)

      # Forward pass
      outputs = model(input_ids, attention_mask=attention_mask, entity_pos_1=entity_1_positions, entity_pos_2=entity_2_positions)
      logits = outputs['logits']

      # Get predicted labels
      preds = torch.argmax(logits, dim=1)

      # Store predictions and true labels
      predictions.extend(preds.cpu().numpy())
      true_labels.extend(labels.cpu().numpy())

    # Calculate accuracy
  accuracy = accuracy_score(true_labels, predictions)
  print(f'Validation Accuracy: {accuracy}')
  predictions = np.array(predictions)
  true_labels = np.array(true_labels)
  mask = true_labels != label_map['Other']
  filtered_predictions = predictions[mask]
  filtered_true_labels = true_labels[mask]

  accuracy = accuracy_score(filtered_true_labels, filtered_predictions)

  f1 = f1_score(filtered_true_labels, filtered_predictions, average='macro')  # or 'micro' or 'weighted'
  print(f'Validation F1 Score: {f1}')


pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Validation Accuracy: 0.8317997791682002
Validation F1 Score: 0.8039543074225779
